In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

seed = 20230926
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.experimental.numpy.random.seed(seed)
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
os.environ["TF_DETERMINISTIC_OPS"] = "1"
# Set a fixed value for the hash seed
os.environ["PYTHONHASHSEED"] = str(seed)
import dolfin as df
import time
from utils import *
from utils_training import *
from matplotlib.lines import Line2D

import seaborn as sns

sns.set_theme()
sns.set_context("paper")
sns.set(rc={"xtick.bottom": True, "ytick.left": True})

In [ ]:
params = np.load("./params.npy")
print(params)
# (pad_mode, pad_size)
tmp = np.load(f"./misfits_val/misfits_0/model_0.npy")
epochs = [10 * (i + 1) for i in range(len(tmp))]
print(f"{len(params) = }   {epochs = }")

In [ ]:
pad_mode, pad_size = params.T
pad_mode = np.unique(pad_mode).astype(int)
pad_size = np.unique(pad_size).astype(int)
colors = list(sns.color_palette("deep").as_hex())
print(colors)

if not (os.path.exists("./images")):
    os.makedirs("./images")

In [ ]:
small_data = False
data = DataLoader(small_data)


In [ ]:
def compute_error(X, Y_pred, Y_true):
    diff = Y_true - Y_pred
    error = tf.sqrt(
        tf.reduce_mean(
            (diff**2) * X[:, :, :, -1],
            axis=[1, 2],
        )
        / tf.reduce_mean(
            ((Y_true) ** 2) * X[:, :, :, -1],
            axis=[1, 2],
        )
    )
    return tf.reduce_mean(error)

In [ ]:
from generate_data import PhiFemSolver

F, Phi, G, params_data = create_FG_numpy(300, 64)
solver = PhiFemSolver(nb_cell=64 - 1, params=params_data)
W_phi_fem = solver.solve_several()
X_test = generate_manual_new_data_numpy(Phi, F / data.max_norm_F, G)
Y_true = W_phi_fem[:, :, :] * X_test[:, :, :, 1] + X_test[:, :, :, 2]

In [ ]:
color_indices = [3, 0, 4, 9]
line_type = ["-s", "--o", ":>", "-x"]

indices_constant = np.where(params[:, 0] == 0)[0]

fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(6, 4), sharex=True, sharey=True)

for i in range(len(indices_constant)):
    index = indices_constant[i]
    pad_mode_i, pad_size_i = params[index]
    if pad_mode_i == 0:
        pad_mode_i = "CONSTANT"
    elif pad_mode_i == 1:
        pad_mode_i = "REFLECT"
    agent = Agent(
        data,
        small_data,
        width=20,
        modes=10,
        nb_layers=4,
        pad_prop=pad_size_i,
        pad_mode=pad_mode_i,
        index_test_case=index,
        loss_level=2,
    )
    L2_errors = []
    for epoch in epochs:
        agent.model.load_weights(
            f"./models/models_{index}/model_{epoch}/model_weights"
        )
        W_pred = agent.model.call(X_test)
        Y_pred = W_pred[:, :, :, 0] * X_test[:, :, :, 1] + G
        L2_error = compute_error(X_test, Y_pred, Y_true)
        L2_errors.append(L2_error)

    ax0.plot(
        epochs,
        L2_errors,
        line_type[i],
        color=colors[color_indices[i]],
        label=f"{2*pad_size_i*100}$\%$",
    )

ax0.set_yscale("log")
ax0.legend(ncol=2, fontsize=12, loc="upper right")

color_indices = [3, 0, 4, 2]
line_type = ["-s", "--o", ":>", "-x"]

indices_reflect = np.where(params[:, 0] == 1)[0]

for i in range(len(indices_reflect)):
    index = indices_reflect[i]
    pad_mode_i, pad_size_i = params[index]
    if pad_mode_i == 0:
        pad_mode_i = "CONSTANT"
    elif pad_mode_i == 1:
        pad_mode_i = "REFLECT"
    agent = Agent(
        data,
        small_data,
        width=20,
        modes=10,
        nb_layers=4,
        pad_prop=pad_size_i,
        pad_mode=pad_mode_i,
        index_test_case=index,
        loss_level=2,
    )
    L2_errors = []
    for epoch in epochs:
        agent.model.load_weights(
            f"./models/models_{index}/model_{epoch}/model_weights"
        )
        W_pred = agent.model.call(X_test)
        Y_pred = W_pred[:, :, :, 0] * X_test[:, :, :, 1] + G
        L2_error = compute_error(X_test, Y_pred, Y_true)
        L2_errors.append(L2_error)

    ax1.plot(
        epochs,
        L2_errors,
        line_type[i],
        color=colors[color_indices[i]],
        label=f"{2*pad_size_i*100}$\%$",
    )

ax1.set_yscale("log")
plt.legend(ncol=2, fontsize=12, loc="upper right")
ax1.set_xlabel("Epochs", fontsize=16)
ax1.set_ylabel(r"                        $L^2$ relative errors", fontsize=16)
plt.tight_layout()
plt.savefig("./images/change_padding.png")
plt.show()

In [ ]:
color_indices = [3, 0, 4, 2]
line_type = ["-s", "--o", ":>", "-x"]

indices = np.where(
    ((params[:, 0] == 0) & (params[:, 1] == 0.1))
    | ((params[:, 0] == 1) & (params[:, 1] == 0.05))
)[0]

plt.figure(figsize=(6, 4))

for i in range(len(indices)):
    index = indices[i]
    pad_mode_i, pad_size_i = params[index]
    if pad_mode_i == 0:
        pad_mode_i = "CONSTANT"
    elif pad_mode_i == 1:
        pad_mode_i = "REFLECT"
    agent = Agent(
        data,
        small_data,
        width=20,
        modes=10,
        nb_layers=4,
        pad_prop=pad_size_i,
        pad_mode=pad_mode_i,
        index_test_case=index,
        loss_level=2,
    )
    L2_errors = []
    for epoch in epochs:
        agent.model.load_weights(
            f"./models/models_{index}/model_{epoch}/model_weights"
        )
        W_pred = agent.model.call(X_test)
        Y_pred = W_pred[:, :, :, 0] * X_test[:, :, :, 1] + G
        L2_error = compute_error(X_test, Y_pred, Y_true)
        L2_errors.append(L2_error)

    if pad_mode_i == "CONSTANT":
        pad_mode_i = "Constant:"
    elif pad_mode_i == "REFLECT":
        pad_mode_i = "Reflect:"
    plt.plot(
        epochs,
        L2_errors,
        line_type[i],
        color=colors[color_indices[i]],
        label=f"{pad_mode_i} {2*pad_size_i*100}$\%$",
    )

plt.yscale("log")
plt.legend(ncol=1, fontsize=16, loc="upper right")
plt.xlabel("Epochs", fontsize=16)
plt.ylabel(r"$L^2$ relative errors", fontsize=16)
plt.tight_layout()
plt.savefig("./images/change_padding_bests.png")
plt.show()